In [174]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.util import *
from nltk.sentiment import SentimentIntensityAnalyzer
from string import punctuation
from collections import Counter
from nltk.corpus import brown
import wikipedia
import pickle

In [38]:
#reading the text

text = None
with open('C:\\Users\\Rita\\Documents\\tech.txt', 'r') as f:
    text = f.read()

In [85]:
#ne_chunk entities
#entity recognition

tokens_1 = nltk.word_tokenize(text)
tagged_1 = nltk.pos_tag(tokens_1)
 
ne_chunked_orig = nltk.ne_chunk(tagged_1, binary=False)
ne_chunked_bin = nltk.ne_chunk(tagged_1, binary=True)

In [176]:
#func for counting

def tokenCounts(tokens):
    counts = Counter(tokens)
    sortedCounts = sorted(counts.items(), key=lambda count:count[1], reverse=True)
    return sortedCounts

In [177]:
#top-10 tokens after pos-tagging

t10_pos=tokenCounts(tagged_1)
t10_pos=t10_pos[:9]

print(t10_pos)

[((',', ','), 693), (('the', 'DT'), 627), (('.', '.'), 549), (('to', 'TO'), 332), (('a', 'DT'), 285), (('of', 'IN'), 273), (('and', 'CC'), 264), (('in', 'IN'), 177), (('for', 'IN'), 132)]


In [86]:
#entity recognition

def extractEntities(ne_chunked):
    data = {}
    for entity in ne_chunked:
        if isinstance(entity, nltk.tree.Tree):
            text = " ".join([word for word, tag in entity.leaves()])
            ent = entity.label()
            data[text] = ent
        else:
            continue
    return data

In [87]:
#entities

entities_orig=extractEntities(ne_chunked_orig)
entities_bin=extractEntities(ne_chunked_bin)


In [179]:
print(entities_bin)

{'Nanoleaf Aurora': 'NE', 'Solar Pool Technologies': 'NE', 'Alexas': 'NE', 'Tearline': 'NE', 'Alexandre Taliercio': 'NE', 'Joe Biden': 'NE', 'Localization Book': 'NE', 'Friedman': 'NE', 'Brexit': 'NE', 'Mastodon': 'NE', 'Verge': 'NE', 'British': 'NE', 'Japanese': 'NE', 'Hajek': 'NE', 'Google Glass': 'NE', 'Model': 'NE', 'Shatner': 'NE', 'HTC Vive': 'NE', 'National Network': 'NE', 'Nvidia': 'NE', 'GIFs': 'NE', 'Oculus': 'NE', 'Play Music': 'NE', 'Schiller': 'NE', 'Las Vegas': 'NE', 'Bill Gates': 'NE', 'Oculus Rift': 'NE', 'South Korea': 'NE', 'Alexandra Fine': 'NE', 'Messenger': 'NE', 'Patricia Lauder': 'NE', 'IoT': 'NE', 'Access': 'NE', 'NES Classic Edition': 'NE', 'Google Play Music All Access': 'NE', 'Guardian': 'NE', 'Touch': 'NE', 'Mac Mini': 'NE', 'Houston': 'NE', 'Helio X20': 'NE', 'James': 'NE', 'Fido': 'NE', 'Canada': 'NE', 'Virtual Sexology': 'NE', 'Bolt EV': 'NE', 'Congress': 'NE', 'Future': 'NE', 'Joe Quirk': 'NE', 'Gear VR': 'NE', 'Indians': 'NE', 'PlayStation VR': 'NE', 'D

In [182]:
#print(tokenCounts(list(entities_bin.keys())))

In [88]:
#custom entities

#more or less nice
grammar = "NP: {((<DT>?<JJ>).*(<NN>?<NNS>?<NNPS>?<NNP>))|((<NN>?<NNS>?<NNPS>?<NNP>).*(<NNP>|<NNPS>))|(<NNP>|<NNPS>)}"
cp = nltk.RegexpParser(grammar)
result = cp.parse(tagged_1)
entities_c=extractEntities(result)
print(entities_c)


{'the only area Facebook': 'NP', 'Alexas': 'NP', 'Sex': 'NP', 'Joe Biden': 'NP', 'Localization Book': 'NP', 'Friedman': 'NP', 'Canada': 'NP', 'Verge': 'NP', 'Hajek': 'NP', 'Google Glass': 'NP', 'Shatner': 'NP', 'HTC Vive': 'NP', '68,000-plus Model': 'NP', 'GIFs': 'NP', 'Play Music': 'NP', 'Bill Gates': 'NP', 'a free Samsung': 'NP', 'Avatars': 'NP', 'Denis Grisak': 'NP', 'like Lego': 'NP', 'Alexandra Fine': 'NP', 'the right direction.вЂќ ReDeTecвЂ™s': 'NP', 'IoT': 'NP', 'Hollywood': 'NP', 'Snapdragon': 'NP', 'Music All': 'NP', 'Mac Mini': 'NP', 'Houston': 'NP', 'United Arab': 'NP', 'Virtual Sexology': 'NP', 'PACCAR': 'NP', 'Congress': 'NP', 'Group Ford': 'NP', 'Fin': 'NP', 'Legend': 'NP', 'HR': 'NP', 'Chu': 'NP', 'Thompson Innovation': 'NP', 'Due': 'NP', 'A': 'NP', 'Solar Pool': 'NP', 'PlayStation VR': 'NP', 'the all-electric Bolt': 'NP', 'Google Play': 'NP', 'Win': 'NP', 'Tuesday': 'NP', 'N': 'NP', 'Pacific': 'NP', 'Gaana': 'NP', 'new Garadget': 'NP', 'Well': 'NP', 'Sweden': 'NP', 'Ube

In [170]:
#function for classification of noun phrase 

def entities_classif(entities):
    classif={}
    for ent in list(entities.keys()):
        try:
            page=wikipedia.page(ent)
        except Exception:
            classif[ent]='MISC'
        else:
            summary=page.summary
            sent = nltk.sent_tokenize(summary)
            if sent!=[]:
                sent=sent[0]
                tok = nltk.word_tokenize(sent)
                tagged= nltk.pos_tag(tok)
                expression="NP: {(<VBZ>+<DT|JJ>+<NN|NNS>)}" #.*<DT>.?<JJ>.*<NN|NNS> <NN|NNS|NNP|NNPS>?<VBZ>?
                cp = nltk.RegexpParser(expression)
                r= cp.parse(tagged)
                e=extractEntities(r)
                if e=={}:
                    classif[ent]='MISC'
                else:
                    classif[ent+' '+list(e.keys())[0]]='NP'
            else:
                classif[ent]="MISC"
    return classif

In [171]:
cl_c=entities_classif(entities_c)
print(cl_c)

C:\Users\Rita\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\Rita\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


{'Verge': 'MISC', 'first Hyperloop is a company': 'NP', 'Denis Grisak': 'MISC', 'Localization Book': 'MISC', 'Friedman': 'MISC', 'Nintendo is a Japanese multinational consumer': 'NP', 'SUV is a vehicle': 'NP', 'Apple Pro is a digital audio workstation': 'NP', 'Ars Technica': 'MISC', 'Google Glass': 'MISC', 'Tearline': 'MISC', 'Shatner': 'MISC', 'Tesla Model is an upcoming all-electric four-door compact luxury': 'NP', 'Google Assistant is an intelligent personal assistant': 'NP', 'Microsoft is an American multinational': 'NP', 'Nanoleaf Aurora is a green technology': 'NP', 'Messenger': 'MISC', 'Play Music is a music': 'NP', 'a premium Samsung is an Italian digital terrestrial television': 'NP', 'IoT': 'MISC', 'Houston': 'MISC', 'early Oculus is a virtual reality': 'NP', 'Apple is a deciduous tree': 'NP', 'immersive VR': 'MISC', 'Future': 'MISC', 'Budget is a quantitative expression': 'NP', 'startup Argo AI is an American multinational': 'NP', 'HR': 'MISC', 'Chu': 'MISC', 'Due': 'MISC', 

In [172]:
cl_orig=entities_classif(entities_orig)
print(cl_orig)

C:\Users\Rita\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\Rita\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


{'Alcatel': 'MISC', 'Verge': 'MISC', 'Canada is a country': 'NP', 'Alexandre Taliercio': 'MISC', 'Patri Friedman is an American libertarian activist': 'NP', 'Denis Grisak': 'MISC', 'Friedman': 'MISC', 'Brexit': 'MISC', 'Mastodon': 'MISC', 'British': 'MISC', 'Japanese': 'MISC', 'Gear VR is a mobile virtual reality': 'NP', 'Google Glass': 'MISC', 'Tearline': 'MISC', 'Shatner': 'MISC', 'Tesla Model is an upcoming all-electric four-door compact luxury': 'NP', 'Nanoleaf Aurora is a green technology': 'NP', 'Revenge is a form': 'NP', 'Medium': 'MISC', 'Raj Nair': 'MISC', 'Tahiti': 'MISC', 'Palmer Luckey is an American entrepreneur': 'NP', 'RAW': 'MISC', 'Politicians is a person': 'NP', 'Helsingborg is a town': 'NP', 'Hernando Chaves': 'MISC', 'Messenger': 'MISC', 'Play Music is a music': 'NP', 'Navigant Research is a specialized expert': 'NP', 'IoT': 'MISC', 'United Arab Emirates is a federal absolute': 'NP', 'Volkswagen Group is a German multinational automotive manufacturing': 'NP', 'Congr

In [175]:
pickle.dump(cl_orig, open( "orig.p", "wb" ) )
pickle.dump(cl_c, open( "clc.p", "wb" ) )

In [ ]:
clorig=pickle.load( open( "orig.p", "rb" ) )
clc=pickle.load( open( "clc.p", "rb" ) )